In [1]:
type Dual
    fun::Real
    der::Real
end
Dual(a::Real) = Dual(a,0)

Dual

In [2]:
#Test
d=Dual(15.1,3)
f(x)=x
f(d)

Dual(15.1,3)

In [3]:
# Esto lo hice para no tener que definir +,-,*,etc entre un Dual y numeros de otro tipo,
# que lo hiciera automaticamente, pero parace no funcionar no se porque.
# http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/
import Base: convert, promote_rule
convert(::Type{Dual},x::Real) = Dual(x,0)
promote_rule(::Type{Dual}, ::Type{Int} ) = Dual
promote_rule(::Type{Dual}, ::Type{Float64} ) = Dual

promote_rule (generic function with 126 methods)

In [4]:
#Test
convert(Dual,1)

Dual(1,0)

In [5]:
#Test
promote(d,1)

(Dual(15.1,3),Dual(1,0))

In [6]:
# los Irracionales dan problemas: Dual(pi,pi)^1 == Dual(pi,pi) da false porque ^1
# los convierte en float64, entonces los convertire en float desde el principio
Dual(pi,pi)

Dual(π = 3.1415926535897...,π = 3.1415926535897...)

In [29]:
Dual(a::Irrational,b::Irrational) = Dual(convert(Float64,a),convert(Float64,b))
Dual(a::Irrational,b::Real) = Dual(convert(Float64,a),convert(Float64,b))
Dual(a::Real,b::Irrational) = Dual(convert(Float64,a),convert(Float64,b))

Dual

In [8]:
Dual(pi,pi)

Dual(3.141592653589793,3.141592653589793)

In [9]:
xdual(x0)=Dual(x0,1)

xdual (generic function with 1 method)

In [10]:
#Test
xdual(1000)

Dual(1000,1)

In [11]:
import Base: +, -, *, /, ^, == 
+(x::Dual,y::Dual) =Dual( x.fun+y.fun , x.der+y.der )
-(x::Dual,y::Dual) =Dual( x.fun-y.fun , x.der-y.der )
*(x::Dual,y::Dual) =Dual( x.fun*y.fun , (x.fun*y.der)+(y.fun*x.der) )
/(x::Dual,y::Dual) =Dual( x.fun/y.fun , (x.der-((x.fun/y.fun)*y.der))/y.fun )

+(x::Dual,y::Real) =Dual( x.fun+y , x.der )
-(x::Dual,y::Real) =Dual( x.fun-y , x.der )
*(x::Dual,y::Real) =x*convert(Dual,y)
/(x::Dual,y::Real) =x/convert(Dual,y)

+(y::Real,x::Dual) =Dual( x.fun+y , x.der )
-(y::Real,x::Dual) =Dual( x.fun-y , x.der )
*(y::Real,x::Dual) =x*convert(Dual,y)
/(y::Real,x::Dual) =x/convert(Dual,y)

^(x::Dual,a::Integer) =Dual( x.fun^a , a*(x.fun^(a-1))*x.der )
^(x::Dual,a::Real) =Dual( x.fun^a , a*(x.fun^(a-1))*x.der )

+(x::Dual) =Dual( +x.fun , +x.der )
-(x::Dual) =Dual( -x.fun , -x.der )

==(x::Dual,y::Dual)=(x.fun,x.der)==(y.fun,y.der)

import Base: print,show
show(io::IO,x::Dual)=print(io, x.fun," + ",x.der,"ε")
print(io::IO,x::Dual)=show(io,x)

print (generic function with 19 methods)

In [12]:
#Test
a=Dual(15,3)
b=Dual(2,5)
c=2
print("a: ",a,"\n")
print("b: ",b,"\n")
print("a+b: ",a+b,"\n")
print("a-b: ",a-b,"\n")
print("a*b: ",a*b,"\n")
print("a/b: ",a/b,"\n")
print("a^2: ",a^2,"\n")
print("a^.5: ",a^.5,"\n")
print("a==b: ",a==b,"\n")
print("a==a: ",a==a,"\n")

a: 15 + 3ε
b: 2 + 5ε
a+b: 17 + 8ε
a-b: 13 + -2ε
a*b: 30 + 81ε
a/b: 7.5 + -17.25ε
a^2: 225 + 90ε


In [13]:
f(x)=x
g(x)=x*x
h(x)=3x^3-2
a=Dual(3,1)
b=xdual(1)

print("a: ",a,"\n")
print("b: ",b,"\n")
print("f(x)=x","\n")
print("g(x)=x*x","\n")
print("h(x)=3x^3-2","\n\n")

print("f(a): ",f(a),"\n")
print("g(a): ",g(a),"\n")
print("h(a): ",h(a),"\n\n")

print("f(b): ",f(b),"\n")
print("g(b): ",g(b),"\n")
print("h(b): ",h(b),"\n\n")

a^.5: 3.872983346207417 + 0.38729833462074165ε
a==b: false
a==a: true
a: 3 + 1ε
b: 1 + 1ε
f(x)=x
g(x)=x*x
h(x)=3x^3-2

f(a): 

In [14]:
# como vemos el valor de h'(1)=9

In [15]:
gtest(x) = (3x^2-8x+5)/(7x^3-1)
gtest(xdual(1))

0.0 + -0.3333333333333333ε

In [ ]:
using Base.Test

In [45]:
A=(0,1,7,0.0,1.0,7.0,pi,-1,-7)
for (a in A)
    for(b in A)
        d=Dual(a,b);
        c=xdual(a)
        if(a!=0)
            @test (d+d-d*d/d)^1==d
            @test (d+a-a*a/a)^1==d
            @test (c+d-d*d/d)^1==c
        else
            @test ((d+d-d)*d)^1==d*d
            @test ((d+a-a)*a)^1==d*a
            @test ((c+d-d)*d)^1==c*d
        end
    end
end